In [1]:
import numpy as np
import pandas as pd
from fractions import Fraction

In [2]:
NUM_PLAYERS = 3
PLAYERS = list(range(1, NUM_PLAYERS+1))
# all possible (giver, receiver) pairs
OUTCOME_PROB = Fraction(1, NUM_PLAYERS * (NUM_PLAYERS-1))  # 1/6
CANDIDATE_MOVES = [(g, r) for idx, g in enumerate(PLAYERS) for r in PLAYERS if g != r]
LOSER_IDX = 0

# Helper functions

## Functional

In [29]:
def applyMove(initState, move):
    """
    Applies the given move to the initial state:
    - initState: an integer triple representing the initial wealth of each player
    - move: a (giver, receiver) pair
    Returns the resulting wealth of the players as a triple.
    """
    g, r = move
    initWealth = list(initState)
    transferAmt = min(initWealth[g-1], initWealth[r-1])  # 0-indexed
    initWealth[g-1] -= transferAmt  # give
    initWealth[r-1] += transferAmt  # receive
    return tuple(initWealth)


def getChildStates(initState):
    """
    Applies all possible moves to the initial state.
    Returns the child states.
    """
    return [applyMove(initState, move) for move in CANDIDATE_MOVES]

def getCoef(initState):
    """
    Returns a pair consisting of:
    - a dictionary of (state:coefficient) pairs, representing the coefficient of the hitting probability 
      for each state in the first-step analysis, starting from the given initial state
    - a rational number representing the constant in the equation.
    """
    # assign all states the individual outcome probability
    childStates = getChildStates(initState)
    stateCoefs = {s:OUTCOME_PROB for s in childStates}
    const = 0
    
    # apply any possible simplifications to the states
    copy = stateCoefs.copy()
    for state in copy:
        # prune the 0 hitting prob states
        if zeroHittingProb(state):
            stateCoefs.pop(state)
        # convert the hitting prob 1 states to constants
        elif (status := alreadyHit(state)):
            const += stateCoefs[state] * status  # 1 for True
            stateCoefs.pop(state)
        # optimsation 1: convert equal wealth states to constants
        elif equalWealth(state):
            const += stateCoefs[state] / NUM_PLAYERS
            stateCoefs.pop(state)
    
    # optimization 2: combine symmetrical states
    copy = list(stateCoefs.copy().keys())
    n = len(copy)
    for i in range(n):
        for j in range(i+1, n):
            s1, s2 = copy[i], copy[j]
            if (symmetricalStates(s1, s2)):
                combinedProb = stateCoefs.pop(s1) + stateCoefs.pop(s2)
                stateCoefs[standardiseState(s1)] = combinedProb
    
    return standardiseStates(stateCoefs), const

def zeroHittingProb(state):
    """
    Returns true if the given state cannot reach the state of Player 1 being the loser, false otherwise.
    """
    return (0 in state) and (state[LOSER_IDX] > 0)

def alreadyHit(state):
    """
    Returns true if the given state reaches the state of Player 1 being the loser wp1, false otherwise.
    """
    return state[LOSER_IDX] == 0

def symmetricalStates(s1, s2):
    """
    Returns true if the two states are symmetrical (undistinguished apart from the loser), false otherwise.
    """
    return s1[LOSER_IDX] == s2[LOSER_IDX] and set(s1[LOSER_IDX+1:]) == set(s2[LOSER_IDX+1:])
    
def equalWealth(s):
    """
    Returns true if all players have the same wealth in the current state (an integer triple), 
    false otherwise.
    """
    return all(amt == s[0] for amt in s)

## Formatting / Representational

In [4]:
from sympy import *
from sympy import nsolve
from sympy.solvers import solve
from ast import literal_eval as make_tuple  # for parsing state as a tuple

In [5]:
def standardiseState(s):
    """
    Returns the standardised form of the state (a triple).
    The convention is to keep the smaller value of the undistinguished players at the front.
    """
    return (s[LOSER_IDX],) + tuple(sorted(s[LOSER_IDX+1:]))

def standardiseStates(stateCoefs):
    """
    Standardises all states (keys) in the dictionary of (state:coefficient) pairs.
    Returns the standardised form of the dictionary.
    """
    return {standardiseState(s):p for s, p in stateCoefs.items()}

def getEquation(initState, stateCoefs, const):
    """
    Converts the dictionary of state:coefficient pairs and constant
    into an equality of the form: h(initState) = coefs * h(states) + const
    """
    # equation will be in the form of human readable equalities (like how we normally write down)
    symbols = [stateToSymbol(s) for s in stateCoefs]
    eqn = Eq(stateToSymbol(initState), getExpr(stateCoefs, const))
    return eqn, symbols
    
def stateToSymbol(state):
    """
    Converts the state (triple) to a Symbol object.
    """
    return Symbol(f'h{state}')

def symbolToState(symbol):
    """
    Converts the Symbol object to a state (triple).
    """
    return make_tuple(str(symbol)[1:])

def getExpr(stateCoefs, const):
    """
    Converts the dictionary of state:coefficient pairs and constant
    into an expression.
    """
    expr = const
    for s in stateCoefs:
        expr = Add(expr, stateCoefs[s] * stateToSymbol(s))
    return expr

def formatEquation(eqn):
    """
    Returns the formatted string for the equation.
    """
    return f'{eqn.lhs} = {eqn.rhs}'

def formatSolutions(sol):
    """
    Returns the solutions as a formatted list (states converted back to triples)
    """
    return [(symbolToState(s), p) for s, p in sol.items()]

## System

In [6]:
import signal

In [7]:
class TimeoutException(Exception):
    """
    Custom exception for timeout.
    """
    pass

def sigalarmHandler(signum, frame):
    """
    Handler function to be called when SIGALRM is received.
    """
    # We get signal!
    raise TimeoutException()

# Main class for the first-step analysis

In [8]:
%run BettingGameSimulation.ipynb

In [9]:
class LoserAnalysis():
    def __init__(self, initState):
        self.initState = standardiseState(initState)
        self.generatedStates = set([initState])
        self.equations = []
        self.symbols = set([stateToSymbol(self.initState)]) # or can encode all generated states at once in the end
    
    def getEquations(self):
        """
        Returns a list of equations (as Equality objects) generated during the first step analysis.
        """
        self.recurseGenerateEqns(self.initState)
        return self.equations
        
    def recurseGenerateEqns(self, initState):
        """
        Recursively generates the first step analysis equations,
        and records the new symbols and equations.
        """
        childStatesProb, const = getCoef(initState)  # state:prob pairs, under unified representation
        newStates = set(childStatesProb).difference(self.generatedStates)
        
        # compute for the current state
        # do this before the recursion to close off the loop
        eqn, symbols = getEquation(initState, childStatesProb, const)
        self.equations.append(eqn)

        # base case: no new states
        if not newStates:
            return

        self.symbols = self.symbols.union(symbols)
        self.generatedStates = self.generatedStates.union(newStates)
        
        # recurse
        for state in newStates:
            self.recurseGenerateEqns(state)
    
    def solveEqns(self):
        """
        Returns the hitting probabilities of all states involved in the analysis,
        as a dictionary of (state symbol : probability) pairs.
        The probabilities are represented as fractions.
        """
        self.getEquations()
        return solve(self.equations, self.symbols)
    
    def solveEqnsFloat(self):
        """
        Similar to `solveEqns()`, but the hitting probabilities are represented as floating point numbers.
        """
        sol = self.solveEqns()
        return {s:float(p) for s, p in sol.items() if sol}
    
    def exportEqns(self, filename):
        """
        Exports the first step analysis equations to a text file, 
        using the provided filename.
        """
        self.getEquations()
        fp = open(filename, 'w')
        for e in self.equations:
            fp.write(formatEquation(e) + '\n')
        fp.close()
    
    def getHittingProb(self):
        """
        Returns the exact hitting probability (as a fraction) of the initial state.
        """
        sol = self.solveEqns()
        return sol[stateToSymbol(self.initState)] if sol else None
    
    def getHittingProbNumerical(self):
        """
        Returns the numerical approximation of the hitting probability.
        """
        return simGame(100000, self.initState, report=False)

## Examples

In [10]:
# Get the approximate solutions
#LoserAnalysis((1,2,3)).solveEqnsApprox()

In [11]:
# Get the exact solutions (as fractions)
#LoserAnalysis((5,7,8)).solveEqns()

In [12]:
# Export the equations
#LoserAnalysis((5,7,8)).exportEqns('578Equations.txt')

# Aggregate states and hitting probs to one file

In [21]:
import csv

In [22]:
def getTargetProbsForStates(minWealth, maxWealth, exact=True, fraction=True):
    """
    Returns hitting probabilities as a dictionary of state : P(Loser = Player 1) pairs, 
    beginning from all possible initial states within the range [`minWealth`, `maxWealth`] (inclusive).
    The hitting probabilities of any intermediate states are NOT included.
    """
    stateProbs = dict()  # states as symbols : hitting prob

    for x in range(minWealth, maxWealth+1):
        for y in range(minWealth, maxWealth+1):
            for z in range(minWealth, maxWealth+1):
                initState = (x, y, z)
                try:
                    if exact:
                        sol = LoserAnalysis(initState).getHittingProb()
                        if not fraction:
                            sol = float(sol)
                    else:
                        sol = LoserAnalysis(initState).getHittingProbNumerical()
                    stateProbs[initState] = sol
                except RecursionError:
                    print(f'Maximum recursion depth exceeded for state {state}')

    return stateProbs

In [16]:
def exportStateProbs(filename, stateProbs, append=False, header=True):
    """
    Creates a CSV file with columns (Initial state, P(Loser = Player 1)).
    The state is an integer triple, and the probability is a rational number.
    - filename: a string representing the filename of the CSV file;
    - stateProbs: a dictionary of state : hitting prob pairs, representing the contents to be exported.
    """
    if append:
        fp = open(filename, 'a')
    else:
        fp = open(filename, 'w')
    writer = csv.writer(fp)
    if header:
        writer.writerow(['Initial state', 'P(Loser = Player 1)'])
    
    writer.writerows(stateProbs.items())
    fp.close()

## Example

In [19]:
# generates the hitting probs for (1,1,1), (1,1,2), ..., (5,5,5)
# probs = getTargetProbsForStates(11,15)
# exportStateProbs('Game1_Player1LoseProbs_1to100.csv', probs, append=True, header=False)
# exportStateProbs('Game2_Player1LoseProbs_1to100_float.csv', {s:float(p) for s,p in probs.items()}, append=True, header=False)

KeyboardInterrupt: 

In [18]:
#LoserAnalysis((5,7,8)).getHittingProb()